# 1. Compute Results Overview Tables

The success of an assembly is evaluated by the computation of metrics in two defined ways: globally through statistics inherent to the complete set of sequences that were assembled, and relative to the replicons present in the sample. 

The computation of the global metrics is done through custom python code from the complete set of sequences assembled. 

The following metrics are computed for the complete and filtered set of assembled sequences, restricted to contigs of length above a specified minimum size: 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Maximum sequence length:** The length of the largest contig in the assembly.
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **Contiguity**
    - **Nx (where 0  < x  ⩽ 100):** Length for which the collection of all assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the assembly
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event

After filtering, the sequences are mapped with the reference replicons and the metrics are computed through custom python code. The metrics are calculated, filtered for a minimum length, for each replicon in the file provided input references. 

- **Contig sizes**
    - **Contigs:** The total number of contigs in the assembly;
    - **Basepairs:** The total number of bases in the assembly;
    - **Number of ‘N’s:** Number of uncalled bases (N's) 
- **COMPASS**
    - **(Breadth of) Coverage:** Ratio of covered sequence on the reference by aligned contigs;
    - **Multiplicity:** Ratio of the length of alignable assembled sequence to covered sequence on the reference;
    - **Validity:** Ratio of the length of the alignable assembled sequence to total basepairs in the aligned contigs;
    - **Parsimony:** Cost of the assembly (multiplicity over validity);
- **Contiguity**
    - **Contiguity:** longest single alignment between the assembly and the reference, relative to the reference length;
    - **NAx (where 0  < x  ⩽ 100):** Length for which the collection of aligned assembled sequences of that length or longer in an assembly covers at least a given percentage of the total length of the reference replicon;
    - **NGx (where 0  < x  ⩽ 100):** Length for which the collection of aligned contigs of that length or longer covers at least a given percentage of the sequence of the reference.
    - **Lx (where 0  < x  ⩽ 100):** Minimal number of contigs that cover x % of the sequence of the reference;
- **Identity**
    - **Identity:** Ratio of identical basepairs in all aligned contigs to the reference;
    - **Lowest identity:** Identity of the lowest scoring contig to the reference.
    - **PLS**: Phred-like score per contig, per assembler.
- **Misassembly**
    - **Misassemblies** - Number of aligned contigs that contain a misassembly event


## Imports

In [21]:
import sys
from plotly.offline import plot
import glob
import fnmatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
import pandas as pd
from itertools import groupby
import csv
import numpy as np

## Global variables

In [87]:
ASSEMBLER_PROCESS_LIST = ["ABYSS", "BCALM2", "GATBMINIAPIPELINE", "METAHIPMER2", "MINIA", "MEGAHIT", "METASPADES", "UNICYCLER", "SPADES",
                          "SKESA", "VELVETOPTIMIZER", "IDBA"]
PROCESS_TO_NAME = {"ABYSS": "AbYSS",
                   "BCALM2": "BCALM2", 
                   "GATBMINIAPIPELINE": "GATBMiniaPipeline",
                   "METAHIPMER2": "MetaHipMer2",
                   "MINIA": "MINIA",
                   "MEGAHIT": "MEGAHIT", 
                   "METASPADES": "metaSPAdes", 
                   "UNICYCLER": "Unicycler", 
                   "SPADES": "SPAdes",
                    "SKESA": "SKESA",
                   "VELVETOPTIMIZER": "VelvetOptimizer",
                   "IDBA": "IDBA-UD"}

log_distributed = ['LHS', 'LNN', 'ERR2935805']
even_distribution = ['EMS', 'LNN', 'ERR2984773']

genomic_assemblers = ['ABySS', 'BCALM2', 'MINIA', 'SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer']
metagenomic_assemblers = ['MetaHipMer2','GATBMiniaPipeline', 'IDBA-UD', 'MEGAHIT', 'metaSPAdes']
single_kmer = ['BCALM2', 'MINIA', 'ABySS']
multiple_kmer = ['SKESA', 'SPAdes', 'Unicycler', 'VelvetOptimizer', 'GATBMiniaPipeline', 
                 'IDBA-UD', 'MEGAHIT', 'metaSPAdes', 'MetaHipMer2']

best_min = ['Ns', 'contigs', 'filtered_Ns', 'filtered_contigs','misassembled contigs','misassembly events']
best_max = ['basepairs','filtered_basepairs','filtered_mapped_reads','filtered_n50','mapped_reads','max_contig','n50']

COLOURS = ['#5876c8', '#58AEC8', '#009392', '#39B185', '#9CCB86', '#E9E29C', '#EEB479', '#E88471', '#CF597E', '#a54765', '#a42a2a', '#835221', 'darkgray']

## Global metrics

### Load data

In [23]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
global_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for line in json_report[sample]['GlobalTable']:
                assembler = line['assembler']
                global_pipeline_metrics_df = global_pipeline_metrics_df.append({'run': stats_run,
                                                                                'sample': sample,
                                                                                'assembler': line['assembler'],
                                                                                'contigs': int(line['original']['contigs']),
                                                                                'basepairs': int(line['original']['basepairs']),
                                                                                'max_contig': int(line['original']['max_contig_size']),
                                                                                'n50': int(line['original']['N50']),
                                                                                'mapped_reads': line['original']['mapped_reads'],
                                                                                'Ns': int(line['original']['Ns']),
                                                                                'misassembled contigs': line['filtered']['misassembled_contigs'],
                                                                                'misassembly events': line['filtered']['misassembly_events'],
                                                                                'filtered_contigs': line['filtered']['contigs'],
                                                                                'filtered_basepairs': line['filtered']['basepairs'],
                                                                                'filtered_n50': line['filtered']['N50'],
                                                                                'filtered_Ns': line['filtered']['Ns'],
                                                                                'filtered_mapped_reads': line['filtered']['mapped_reads'],
                                                                               },
                                                                               ignore_index=True)
global_pipeline_metrics_df['distribution'] = np.where(global_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
global_pipeline_metrics_df['type'] = np.where(global_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
global_pipeline_metrics_df['algorythm'] = np.where(global_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50', 'filtered_n50','misassembled contigs', 'misassembly events']] = global_pipeline_metrics_df[['contigs','basepairs','max_contig', 'Ns','n50','filtered_n50','misassembled contigs', 'misassembly events']].apply(pd.to_numeric)

Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...
Processing pipeline_report_tables.json data from run1...
Processing pipeline_report_tables.json data from run3...
Processing pipeline_report_tables.json data from run2...


### Preview data

In [24]:
global_pipeline_metrics_df.dtypes

Ns                       float64
assembler                 object
basepairs                float64
contigs                  float64
filtered_Ns              float64
filtered_basepairs       float64
filtered_contigs         float64
filtered_mapped_reads    float64
filtered_n50             float64
mapped_reads             float64
max_contig               float64
misassembled contigs     float64
misassembly events       float64
n50                      float64
run                       object
sample                    object
distribution              object
type                      object
algorythm                 object
dtype: object

In [25]:
global_pipeline_metrics_df

,Ns,assembler,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50,run,sample,distribution,type,algorythm
0,1345.0,ABySS,20132244.0,44619.0,400.0,13573790.0,1211.0,98.901529,192306.0,99.851184,851314.0,0.0,0.0,9569.0,run3,LNN,Log,Genomic,Single k-mer De Bruijn graph
1,0.0,BCALM2,19635512.0,44825.0,0.0,13397815.0,784.0,97.222124,31436.0,98.306950,163543.0,0.0,0.0,18682.0,run3,LNN,Log,Genomic,Single k-mer De Bruijn graph
2,0.0,GATBMiniaPipeline,13911245.0,693.0,0.0,13658336.0,219.0,98.807198,140561.0,99.719684,632215.0,1.0,3.0,136741.0,run3,LNN,Log,Metagenomic,Multiple k-mer De Bruijn graph
3,0.0,IDBA-UD,14324920.0,1987.0,0.0,13642314.0,261.0,98.951939,108043.0,99.694880,585610.0,2.0,2.0,105808.0,run3,LNN,Log,Metagenomic,Multiple k-mer De Bruijn graph
4,0.0,MEGAHIT,14402555.0,1842.0,0.0,13704539.0,172.0,99.686862,210993.0,99.765145,1080013.0,1.0,2.0,203548.0,run3,LNN,Log,Metagenomic,Multiple k-mer De Bruijn graph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0.0,MINIA,30648944.0,16748.0,0.0,28218335.0,3574.0,92.157082,20782.0,95.935753,161812.0,0.0,0.0,17967.0,run2,EMS,Even,Genomic,Single k-mer De Bruijn graph
212,0.0,SKESA,30249132.0,1269.0,0.0,30064811.0,891.0,97.237351,74138.0,97.952533,632013.0,9.0,18.0,73839.0,run2,EMS,Even,Genomic,Multiple k-mer De Bruijn graph
213,0.0,SPAdes,30664883.0,2628.0,0.0,30354786.0,412.0,98.283354,208991.0,98.700052,850176.0,0.0,0.0,206515.0,run2,EMS,Even,Genomic,Multiple k-mer De Bruijn graph
214,0.0,Unicycler,30301867.0,656.0,0.0,30188943.0,334.0,97.609864,245050.0,98.457135,1303875.0,0.0,0.0,245050.0,run2,EMS,Even,Genomic,Multiple k-mer De Bruijn graph


### Save data

In [26]:
global_pipeline_metrics_df.to_csv("Tables/Results/Global metrics.csv")

### Global stats per assembler - per sample

#### Values

In [27]:
best_stats_dfs = []

for sample in global_pipeline_metrics_df['sample'].unique():
    print(sample)
    stats_per_assembler = pd.DataFrame()
    best_stats_per_assembler = pd.DataFrame()
    for assembler in global_pipeline_metrics_df['assembler'].unique():
        
        row = {'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        
        best_row = {'Assembler': assembler}
        
        describe_df = global_pipeline_metrics_df[(global_pipeline_metrics_df['assembler'] == assembler) & (global_pipeline_metrics_df['sample'] == sample)].describe()

        for column in describe_df.columns:
            mean = describe_df.loc['mean',column]
            minimum = describe_df.loc['min',column]
            maximum = describe_df.loc['max',column]
            
            row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
            
            if column in best_min:
                best_row[column] = minimum
            else:
                best_row[column] = maximum
                
        stats_per_assembler=stats_per_assembler.append(row, ignore_index=True)
        best_stats_per_assembler=best_stats_per_assembler.append(best_row, ignore_index=True)
    
    stats_per_assembler = stats_per_assembler.set_index(['Assembler','Algorithm', 'Type'])
    best_stats_per_assembler = best_stats_per_assembler.set_index('Assembler')
    display(stats_per_assembler)
    display(best_stats_per_assembler)
    stats_per_assembler.to_csv("Tables/Results/Global metrics per assembler - {}.csv".format(sample))
    best_stats_per_assembler.to_csv("Tables/Results/Best Global metrics per assembler - {}.csv".format(sample))
    best_stats_dfs.append([sample, best_stats_per_assembler])

LNN


,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,1345.67 [1345;1347],20132373.0 [20132244;20132631],44619.67 [44619;44621],400.0 [400;400],13573771.33 [13573734;13573790],1211.33 [1211;1212],98.9 [98;98],192306.0 [192306;192306],99.85 [99;99],851314.0 [851314;851314],0.0 [0;0],0.0 [0;0],9569.0 [9569;9569]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],19635512.0 [19635512;19635512],44825.0 [44825;44825],0.0 [0;0],13397815.0 [13397815;13397815],784.0 [784;784],97.22 [97;97],31436.0 [31436;31436],98.31 [98;98],163543.0 [163543;163543],0.0 [0;0],0.0 [0;0],18682.0 [18682;18682]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],13911109.33 [13910838;13911245],693.0 [693;693],0.0 [0;0],13658336.0 [13658336;13658336],219.0 [219;219],98.81 [98;98],140561.0 [140561;140561],99.72 [99;99],632215.0 [632215;632215],1.0 [1;1],3.0 [3;3],136741.0 [136741;136741]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],14324862.33 [14324747;14324920],1987.0 [1987;1987],0.0 [0;0],13642314.0 [13642314;13642314],261.0 [261;261],98.95 [98;98],108043.0 [108043;108043],99.69 [99;99],585610.0 [585610;585610],2.0 [2;2],2.0 [2;2],105808.0 [105808;105808]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],14402555.0 [14402555;14402555],1842.0 [1842;1842],0.0 [0;0],13704539.0 [13704539;13704539],172.0 [172;172],99.69 [99;99],210993.0 [210993;210993],99.77 [99;99],1080013.0 [1080013;1080013],1.0 [1;1],2.0 [2;2],203548.0 [203548;203548]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],15514698.0 [15514698;15514698],6700.0 [6700;6700],0.0 [0;0],13692577.0 [13692577;13692577],139.0 [139;139],99.59 [99;99],239185.0 [239185;239185],99.8 [99;99],850099.0 [850099;850099],1.0 [1;1],3.0 [3;3],220609.0 [220609;220609]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],16401048.67 [16401028;16401090],15000.33 [15000;15001],0.0 [0;0],13501499.33 [13501498;13501500],762.0 [762;762],97.53 [97;97],33550.0 [33550;33550],99.01 [99;99],177038.0 [177038;177038],2.0 [2;2],4.0 [4;4],26474.0 [26474;26474]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],8202941.0 [8202941;8202941],9514.0 [9514;9514],0.0 [0;0],4470092.0 [4470092;4470092],1433.0 [1433;1433],91.77 [91;91],7385.0 [7385;7385],95.1 [95;95],59272.0 [59272;59272],12.0 [12;12],23.0 [23;23],1155.0 [1155;1155]


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,1345.0,20132631.0,44619.0,400.0,13573790.0,1211.0,98.901625,192306.0,99.851429,851314.0,0.0,0.0,9569.0
BCALM2,0.0,19635512.0,44825.0,0.0,13397815.0,784.0,97.222124,31436.0,98.307446,163543.0,0.0,0.0,18682.0
GATBMiniaPipeline,0.0,13911245.0,693.0,0.0,13658336.0,219.0,98.807198,140561.0,99.719684,632215.0,1.0,3.0,136741.0
IDBA-UD,0.0,14324920.0,1987.0,0.0,13642314.0,261.0,98.951945,108043.0,99.694880,585610.0,2.0,2.0,105808.0
MEGAHIT,0.0,14402555.0,1842.0,0.0,13704539.0,172.0,99.686868,210993.0,99.765146,1080013.0,1.0,2.0,203548.0
MetaHipMer2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
metaSPAdes,0.0,15514698.0,6700.0,0.0,13692577.0,139.0,99.591855,239185.0,99.798003,850099.0,1.0,3.0,220609.0
MINIA,0.0,16401090.0,15000.0,0.0,13501500.0,762.0,97.535204,33550.0,99.011021,177038.0,2.0,4.0,26474.0
SKESA,0.0,8202941.0,9514.0,0.0,4470092.0,1433.0,91.766425,7385.0,95.101330,59272.0,12.0,23.0,1155.0


ERR2935805


,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,9.0 [9;9],78727988.67 [78727717;78728532],677855.33 [677853;677860],0.0 [0;0],3150980.0 [3150980;3150980],356.0 [356;356],79.03 [79;79],18568.0 [18568;18568],91.83 [91;91],88665.0 [88665;88665],0.0 [0;0],0.0 [0;0],101.0 [101;101]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],315567452.0 [315567452;315567452],8035706.0 [8035706;8035706],0.0 [0;0],657074.0 [657074;657074],504.0 [504;504],0.1 [0;0],1262.0 [1262;1262],1.43 [1;1],3093.0 [3093;3093],0.0 [0;0],0.0 [0;0],35.0 [35;35]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],17140207.33 [17126362;17147130],6776.33 [6776;6777],0.0 [0;0],13905380.67 [13891328;13912407],603.0 [603;603],88.9 [88;88],90393.0 [90095;90989],89.4 [89;89],542997.0 [542997;542997],0.67 [0;1],1.33 [0;2],55722.33 [55204;56759]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],23587129.0 [23586883;23587252],14155.67 [14155;14156],0.0 [0;0],17534580.0 [17534580;17534580],2725.0 [2725;2725],89.23 [89;89],113853.0 [113853;113853],89.8 [89;89],632213.0 [632213;632213],3.0 [3;3],5.0 [5;5],51161.0 [51161;51161]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],24498566.0 [24498566;24498566],14172.0 [14172;14172],0.0 [0;0],18552996.0 [18552996;18552996],3044.0 [3044;3044],89.5 [89;89],210993.0 [210993;210993],89.76 [89;89],768410.0 [768410;768410],1.0 [1;1],2.0 [2;2],82249.0 [82249;82249]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],15029445.33 [15029183;15029970],11970.67 [11966;11973],0.0 [0;0],10257182.33 [10253731;10264085],5033.33 [5031;5038],73.91 [73;73],2174.67 [2170;2177],84.35 [84;84],14760.0 [14760;14760],0.0 [0;0],0.0 [0;0],1515.67 [1515;1516]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],27822010.0 [27822010;27822010],26282.0 [26282;26282],0.0 [0;0],18792639.0 [18792639;18792639],3392.0 [3392;3392],89.37 [89;89],87983.0 [87983;87983],89.93 [89;89],622107.0 [622107;622107],3.0 [3;3],7.0 [7;7],3674.0 [3674;3674]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],20758237.33 [20757924;20758864],37287.33 [37281;37300],0.0 [0;0],13447286.0 [13446596;13448666],1464.0 [1463;1466],84.24 [84;84],18204.67 [18038;18288],87.68 [87;87],163543.0 [163543;163543],2.0 [2;2],3.0 [3;3],7485.0 [7485;7485]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],8818742.0 [8818742;8818742],8822.0 [8822;8822],0.0 [0;0],5801439.0 [5801439;5801439],2889.0 [2889;2889],62.71 [62;62],2148.0 [2148;2148],78.66 [78;78],29155.0 [29155;29155],0.0 [0;0],0.0 [0;0],1431.0 [1431;1431]


...

,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,98.67 [0;149],61139451.0 [0;91720949],331860.0 [0;497797],4.0 [0;7],20429844.33 [0;30657287],800.33 [0;1205],53.74 [0;80],35189.0 [0;52869],56.42 [0;84],198658.0 [0;298093],1.33 [0;2],3.33 [0;6],92.67 [0;139]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],170402348.0 [170402348;170402348],3508016.0 [3508016;3508016],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],0.0 [0;0],8.15 [8;8],994.0 [994;994],0.0 [0;0],0.0 [0;0],49.0 [49;49]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],32809889.0 [32809112;32810830],5398.33 [5397;5400],0.0 [0;0],30564350.0 [30564302;30564396],761.0 [761;761],82.23 [82;82],113162.0 [113162;113162],83.14 [83;83],768462.0 [768462;768462],1.0 [1;1],2.0 [2;2],102115.0 [102115;102115]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],39776993.0 [39773020;39779384],20182.33 [20164;20194],0.0 [0;0],31562689.67 [31562689;31562690],2654.0 [2654;2654],81.31 [81;81],29134.0 [29134;29134],83.67 [83;83],303329.0 [303329;303329],4.0 [4;4],7.0 [7;7],20993.0 [20993;20993]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],41166407.0 [41166407;41166407],20614.0 [20614;20614],0.0 [0;0],32021766.0 [32021766;32021766],1447.0 [1447;1447],82.83 [82;82],149841.0 [149841;149841],84.22 [84;84],1205839.0 [1205839;1205839],15.0 [15;15],31.67 [31;32],88308.0 [88308;88308]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],13179761.33 [13174826;13184199],19356.0 [19351;19361],0.0 [0;0],1600962.67 [1599587;1601911],1309.0 [1308;1310],3.96 [3;3],1176.0 [1175;1178],25.5 [25;25],2964.0 [2964;2964],0.0 [0;0],0.0 [0;0],658.67 [658;659]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],48964620.0 [48964620;48964620],61644.0 [61644;61644],0.0 [0;0],31967753.0 [31967753;31967753],1689.0 [1689;1689],82.04 [82;82],95101.0 [95101;95101],84.33 [84;84],473277.0 [473277;473277],2.0 [2;2],3.0 [3;3],42332.0 [42332;42332]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],41186649.33 [41185696;41187744],67991.0 [67974;68012],0.0 [0;0],28277982.0 [28276538;28279162],3980.33 [3978;3983],74.36 [74;74],15886.33 [15817;15930],80.65 [80;80],117945.0 [117945;117945],0.0 [0;0],0.0 [0;0],7090.67 [7060;7106]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],30430077.0 [30430077;30430077],1352.0 [1352;1352],0.0 [0;0],30211740.0 [30211740;30211740],846.0 [846;846],80.51 [80;80],77870.0 [77870;77870],81.25 [81;81],446163.0 [446163;446163],0.0 [0;0],0.0 [0;0],76924.0 [76924;76924]


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,0.0,91720949.0,0.0,0.0,30657287.0,0.0,80.631071,52869.0,84.624746,298093.0,0.0,0.0,139.0
BCALM2,0.0,170402348.0,3508016.0,0.0,0.0,0.0,0.000000,0.0,8.149572,994.0,0.0,0.0,49.0
GATBMiniaPipeline,0.0,32810830.0,5397.0,0.0,30564396.0,761.0,82.225440,113162.0,83.144413,768462.0,1.0,2.0,102115.0
IDBA-UD,0.0,39779384.0,20164.0,0.0,31562690.0,2654.0,81.309318,29134.0,83.671484,303329.0,4.0,7.0,20993.0
MEGAHIT,0.0,41166407.0,20614.0,0.0,32021766.0,1447.0,82.834929,149841.0,84.223732,1205839.0,15.0,31.0,88308.0
MetaHipMer2,0.0,13184199.0,19351.0,0.0,1601911.0,1308.0,3.970853,1178.0,25.512982,2964.0,0.0,0.0,659.0
metaSPAdes,0.0,48964620.0,61644.0,0.0,31967753.0,1689.0,82.040817,95101.0,84.326108,473277.0,2.0,3.0,42332.0
MINIA,0.0,41187744.0,67974.0,0.0,28279162.0,3978.0,74.366709,15930.0,80.648560,117945.0,0.0,0.0,7106.0
SKESA,0.0,30430077.0,1352.0,0.0,30211740.0,846.0,80.508554,77870.0,81.254167,446163.0,0.0,0.0,76924.0


ENN


,,,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,Algorithm,Type,,,,,,,,,,,,,
ABySS,Single k-mer De Bruijn graph,Genomic,3881.0 [3879;3884],31510320.33 [31503127;31513997],4772.33 [4769;4778],2328.0 [2328;2328],30742562.33 [30734590;30746773],1823.33 [1822;1825],98.92 [98;98],89282.0 [89282;89282],99.79 [99;99],952617.33 [952616;952618],7.33 [7;8],12.67 [12;14],86573.0 [86573;86573]
BCALM2,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],31047581.0 [31047581;31047581],25377.0 [25377;25377],0.0 [0;0],28174748.0 [28174748;28174748],3575.0 [3575;3575],92.59 [92;92],20700.0 [20700;20700],96.52 [96;96],161812.0 [161812;161812],0.0 [0;0],0.0 [0;0],17395.0 [17395;17395]
GATBMiniaPipeline,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30440762.0 [30440762;30440762],1100.0 [1100;1100],0.0 [0;0],30273320.0 [30273320;30273320],814.0 [814;814],98.46 [98;98],90988.0 [90988;90988],99.34 [99;99],632215.0 [632215;632215],3.0 [3;3],6.0 [6;6],90346.0 [90346;90346]
IDBA-UD,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30390566.0 [30390566;30390566],1822.0 [1822;1822],0.0 [0;0],30130413.0 [30130413;30130413],1279.0 [1279;1279],97.92 [97;97],65108.0 [65108;65108],99.34 [99;99],585610.0 [585610;585610],17.33 [17;18],34.67 [34;36],63725.0 [63725;63725]
MEGAHIT,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30545419.0 [30545419;30545419],814.0 [814;814],0.0 [0;0],30420265.0 [30420265;30420265],525.0 [525;525],99.39 [99;99],179870.0 [179870;179870],99.69 [99;99],632274.0 [632274;632274],9.0 [9;9],18.0 [18;18],178381.0 [178381;178381]
MetaHipMer2,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30378682.33 [30378611;30378729],882.67 [882;883],0.0 [0;0],30264906.33 [30264835;30264953],718.67 [718;719],97.95 [97;97],102049.0 [102049;102049],98.73 [98;98],468359.0 [468359;468359],0.0 [0;0],0.0 [0;0],99990.0 [99990;99990]
metaSPAdes,Multiple k-mer De Bruijn graph,Metagenomic,0.0 [0;0],30453778.0 [30453778;30453778],640.0 [640;640],0.0 [0;0],30383186.0 [30383186;30383186],359.0 [359;359],98.92 [98;98],238611.0 [238611;238611],99.53 [99;99],1036942.0 [1036942;1036942],2.0 [2;2],6.0 [6;6],238611.0 [238611;238611]
MINIA,Single k-mer De Bruijn graph,Genomic,0.0 [0;0],30641342.33 [30641333;30641357],16570.0 [16570;16570],0.0 [0;0],28205937.67 [28205937;28205939],3589.0 [3589;3589],92.93 [92;92],20628.0 [20628;20628],97.19 [97;97],161812.0 [161812;161812],0.0 [0;0],0.0 [0;0],17820.0 [17820;17820]
SKESA,Multiple k-mer De Bruijn graph,Genomic,0.0 [0;0],30110758.0 [30110758;30110758],2322.0 [2322;2322],0.0 [0;0],29835112.0 [29835112;29835112],1726.0 [1726;1726],97.42 [97;97],43168.0 [43168;43168],98.31 [98;98],630384.0 [630384;630384],10.0 [10;10],19.0 [19;19],42115.0 [42115;42115]


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,3879.0,31513997.0,4769.0,2328.0,30746773.0,1822.0,98.923812,89282.0,99.787690,952618.0,7.0,12.0,86573.0
BCALM2,0.0,31047581.0,25377.0,0.0,28174748.0,3575.0,92.594028,20700.0,96.523967,161812.0,0.0,0.0,17395.0
GATBMiniaPipeline,0.0,30440762.0,1100.0,0.0,30273320.0,814.0,98.462815,90988.0,99.340345,632215.0,3.0,6.0,90346.0
IDBA-UD,0.0,30390566.0,1822.0,0.0,30130413.0,1279.0,97.922883,65108.0,99.343697,585610.0,17.0,34.0,63725.0
MEGAHIT,0.0,30545419.0,814.0,0.0,30420265.0,525.0,99.388110,179870.0,99.693845,632274.0,9.0,18.0,178381.0
MetaHipMer2,0.0,30378729.0,882.0,0.0,30264953.0,718.0,97.946876,102049.0,98.726583,468359.0,0.0,0.0,99990.0
metaSPAdes,0.0,30453778.0,640.0,0.0,30383186.0,359.0,98.917250,238611.0,99.530051,1036942.0,2.0,6.0,238611.0
MINIA,0.0,30641357.0,16570.0,0.0,28205939.0,3589.0,92.933709,20628.0,97.188180,161812.0,0.0,0.0,17820.0
SKESA,0.0,30110758.0,2322.0,0.0,29835112.0,1726.0,97.424467,43168.0,98.313391,630384.0,10.0,19.0,42115.0


#### rank

In [204]:
rank_dfs = []
for sample,df in best_stats_dfs:
    print(sample)
    #display(df)
    
    rank_df = pd.DataFrame(index=df.index)
    
    for column in df.columns:
        
        if "basepairs" in column:
            target = 30946587
            diffs = []
            for item in df[column]:
                diff = abs(item - target)
                diffs.append(item)
            sorted_column = sorted(diffs)
            
            rank_list = []
            for item in diffs:
                rank_list.append(sorted_column.index(item) + 1)
            
        else:
            sorted_order = True if column in best_min else False
            sorted_column = list(df[column].sort_values(ascending=sorted_order))

            rank_list = []
            for item in df[column]:
                rank_list.append(sorted_column.index(item) + 1)
        
        rank_df[column] = rank_list 
    display(rank_df)
    rank_df.to_csv("Tables/Results/Rank Global metrics per assembler - {}.csv".format(sample))
    rank_dfs.append([sample, rank_df])
            

LNN


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,11,12,11,12,6,11,5,5,1,2,1,1,10
BCALM2,1,10,12,1,4,10,10,10,10,10,1,1,9
GATBMiniaPipeline,1,4,3,1,9,6,6,6,4,7,7,9,4
IDBA-UD,1,5,5,1,8,8,4,8,6,8,10,7,7
MEGAHIT,1,6,4,1,12,4,1,3,3,1,7,7,3
MetaHipMer2,1,1,1,1,1,1,12,12,12,12,1,1,12
metaSPAdes,1,7,6,1,11,3,2,2,2,3,7,9,2
MINIA,1,9,9,1,5,9,9,9,8,9,10,11,8
SKESA,1,2,8,1,2,12,11,11,11,11,12,12,11


ERR2935805


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,12,11,11,1,3,3,7,7,1,8,1,1,10
BCALM2,1,12,12,1,2,4,11,11,11,11,1,1,12
GATBMiniaPipeline,1,4,2,1,8,5,4,4,5,6,1,1,3
IDBA-UD,1,6,5,1,9,7,3,3,3,4,10,10,4
MEGAHIT,1,7,6,1,10,9,1,2,4,3,8,8,2
MetaHipMer2,1,3,4,1,6,12,8,9,9,10,1,1,8
metaSPAdes,1,8,7,1,11,10,2,5,2,5,10,11,6
MINIA,1,5,9,1,7,6,6,8,7,7,9,9,5
SKESA,1,1,3,1,4,8,9,10,10,9,1,1,9


LHS


...

,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,11,11,11,11,12,4,5,5,6,5,8,7,5
BCALM2,1,12,12,1,1,12,12,12,12,12,1,1,12
GATBMiniaPipeline,1,6,4,1,8,6,4,6,2,7,6,6,6
IDBA-UD,1,4,7,1,5,9,7,9,5,10,12,12,9
MEGAHIT,1,8,5,1,11,5,1,4,1,6,11,11,4
MetaHipMer2,1,3,3,1,7,7,8,7,8,9,1,1,7
metaSPAdes,1,5,2,1,10,2,2,3,3,2,6,7,2
MINIA,1,9,10,1,2,11,11,11,10,11,1,1,11
SKESA,1,1,6,1,4,8,9,8,9,8,10,10,8


ERR2984773


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,1,11,1,1,8,1,7,7,1,8,1,1,10
BCALM2,1,12,12,1,1,1,12,12,11,12,1,1,12
GATBMiniaPipeline,1,4,4,1,7,5,2,3,6,3,7,7,2
IDBA-UD,1,5,6,1,9,11,5,8,5,7,10,10,7
MEGAHIT,1,6,7,1,11,8,1,2,3,1,12,12,3
MetaHipMer2,1,1,5,1,3,7,10,10,10,10,1,1,9
metaSPAdes,1,9,9,1,10,9,3,5,2,5,8,8,6
MINIA,1,7,10,1,4,12,9,9,9,9,1,1,8
SKESA,1,2,3,1,5,6,8,6,8,6,1,1,4


ENN


,Ns,basepairs,contigs,filtered_Ns,filtered_basepairs,filtered_contigs,filtered_mapped_reads,filtered_n50,mapped_reads,max_contig,misassembled contigs,misassembly events,n50
Assembler,,,,,,,,,,,,,
ABySS,11,12,9,11,12,9,2,7,1,2,8,9,7
BCALM2,1,11,12,1,1,11,12,11,12,11,1,1,12
GATBMiniaPipeline,1,5,5,1,8,6,5,6,6,7,6,5,6
IDBA-UD,1,4,7,1,6,7,8,8,5,9,12,12,8
MEGAHIT,1,9,2,1,11,4,1,4,2,6,10,10,4
MetaHipMer2,1,3,3,1,7,5,6,5,8,10,1,1,5
metaSPAdes,1,7,1,1,10,1,3,1,3,1,5,5,1
MINIA,1,10,11,1,2,12,11,12,10,11,1,1,11
SKESA,1,1,8,1,4,8,9,9,9,8,11,11,9


#### radar plots

In [205]:
import plotly.graph_objects as go

for sample, df in rank_dfs:

    categories = ['contigs','filtered_contigs','basepairs','filtered_basepairs', 'max_contig', 'n50','filtered_n50', 'mapped_reads','filtered_mapped_reads', 'Ns','filtered_Ns','misassembled contigs', 'misassembly events']
    categories_original =  ['contigs','basepairs', 'max_contig', 'n50', 'mapped_reads', 'Ns'] 
    categories_original_renamed =  ['Contigs','Basepairs', 'Largest contig', 'N50', '% Mapped reads', 'Uncalled bases'] 
    categories_filtered = ['filtered_contigs', 'filtered_basepairs', 'filtered_n50', 'filtered_mapped_reads','filtered_Ns', 'misassembled contigs', 'misassembly events']
    categories_filtered_renamed = ['Contigs', 'Basepairs', 'N50', '% Mapped reads','Uncalled bases', 'Misassembled contigs', 'Misassembly events']

    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}]*2], subplot_titles=('Original', 'Filtered'))
    
    i= 0
    
    for assembler, row in df.iterrows():
        
        row_filtered = []
        row_original = []
        
        for col in categories_original:
            row_original.append(row.loc[:].at[col])
            
        for col in categories_filtered:
            row_filtered.append(row.loc[:].at[col])

        fig.add_trace(go.Scatterpolar(
          r=row_original,
          theta=categories_original_renamed,
          name=assembler, line=dict(color=COLOURS[i]), showlegend=True), col=1, row=1)
        
        fig.add_trace(go.Scatterpolar(
          r=row_filtered,
          theta=categories_filtered_renamed,
          name=assembler, line=dict(color=COLOURS[i]), showlegend=False), col=2, row=1)
        
        i+=1
            
    fig.update_layout(polar=dict(radialaxis=dict(visible=True,
                                                 range=[1, 12],
                                                 autorange="reversed",
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                angularaxis_categoryarray = categories_original_renamed,
                                radialaxis_angle = -22.5),
                     polar2=dict(radialaxis=dict(visible=True,
                                                 range=[1, 12],
                                                 autorange="reversed",
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                angularaxis_categoryarray = categories_filtered_renamed,
                                radialaxis_angle = -22.5))
    fig.update_layout(title=sample)

    fig.show()



In [207]:
import plotly.graph_objects as go

for sample, df in rank_dfs:

    categories = ['contigs','filtered_contigs','basepairs','filtered_basepairs', 'max_contig', 'n50','filtered_n50', 'mapped_reads','filtered_mapped_reads', 'Ns','filtered_Ns','misassembled contigs', 'misassembly events']
    categories_original =  ['contigs','basepairs', 'max_contig', 'n50', 'mapped_reads', 'Ns'] 
    categories_original_renamed =  ['Contigs','Basepairs', 'Largest contig', 'N50', '% Mapped reads', 'Uncalled bases'] 
    categories_filtered = ['filtered_contigs', 'filtered_basepairs', 'filtered_n50', 'filtered_mapped_reads','filtered_Ns', 'misassembled contigs', 'misassembly events']
    categories_filtered_renamed = ['Contigs', 'Basepairs', 'N50', '% Mapped reads','Uncalled bases', 'Misassembled contigs', 'Misassembly events']

    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}]*2], subplot_titles=('Original', 'Filtered'))
    
    COLOURS_ORIGINAL = ['#5876c8', '#009392', '#9CCB86', '#EEB479', '#E88471', '#CF597E']
    COLOURS_FILTERED = ['#5876c8', '#009392', '#EEB479', '#E88471', '#CF597E', '#a54765', '#835221']

    i = 0
    j = 0
    assembler_list = list(df.index)
    
    for column in categories:
                
        row_all = list(df[column])
        

        if column in categories_original:
            name = categories_original_renamed[categories_original.index(column)]
            fig.add_trace(go.Scatterpolar(r=row_all,
                                          theta=assembler_list,
                                          name=name, 
                                          mode='markers', 
                                          marker=dict(color=COLOURS_ORIGINAL[i], size=12), 
                                          marker_line_color="black", 
                                          marker_line_width=2,
                                          opacity=0.4,showlegend=True),
                          row=1, col=1) 
            i+=1
            
        elif column in categories_filtered:
            name = categories_filtered_renamed[categories_filtered.index(column)]
            showlegend = True if column in ['misassembled contigs', 'misassembly events'] else False
            fig.add_trace(go.Scatterpolar(r=row_all,
                                          theta=assembler_list,
                                          name=name, 
                                          mode='markers', 
                                          marker=dict(color=COLOURS_FILTERED[j], size=12),
                                          marker_line_color="black", 
                                          marker_line_width=2, 
                                          opacity=0.4,showlegend=showlegend),
                         row=1, col=2)
            j+=1
        
    
    fig.update_layout(polar=dict(radialaxis=dict(visible=True,
                                                 range=[1, 12],
                                                 autorange="reversed",
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                angularaxis_categoryarray = assembler_list + [' '],
                                radialaxis_angle = -22.5),
                     polar2=dict(radialaxis=dict(visible=True,
                                                 range=[1, 12],
                                                 autorange="reversed",
                                                 linewidth = 2,
                                                 linecolor="black",
                                                 gridcolor = "#DCDCDC"), 
                                 hole=1/12, bgcolor='rgb(255,255,255)',
                                angularaxis=dict(linecolor="black"),
                                angularaxis_categoryarray = assembler_list + [' '],
                                radialaxis_angle = -22.5))
    fig.update_layout(title=sample)

    fig.show()



### Global statistics per assembler type

In [ ]:
stats_per_assembler_type = pd.DataFrame()
for assembly_type in global_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.type == assembly_type) & (global_pipeline_metrics_df.algorythm == "Multiple k-mer De Bruijn graph")].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler_type=stats_per_assembler_type.append(row, ignore_index=True)
stats_per_assembler_type = stats_per_assembler_type.set_index('Assembler Type')
stats_per_assembler_type.to_csv("Tables/Results/Global metrics per assembler type multiple kmer.csv")

In [ ]:
stats_per_assembler_type

### Global stats per assembly algorithm

In [ ]:
stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in global_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=global_pipeline_metrics_df[global_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2), int(minimum), int(maximum))
    stats_per_assembler_algo=stats_per_assembler_algo.append(row, ignore_index=True)
stats_per_assembler_algo = stats_per_assembler_algo.set_index('Assembler Algorithm')
stats_per_assembler_algo.to_csv("Tables/Results/Global metrics per assembler algorithm.csv")

In [ ]:
stats_per_assembler_algo

### Global stats per distribution

In [ ]:
for distribution in global_pipeline_metrics_df.distribution.unique():
    stats_per_distribution = pd.DataFrame()
    for assembler in global_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].type.unique()[0],
             'Algorithm': global_pipeline_metrics_df[global_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=global_pipeline_metrics_df[(global_pipeline_metrics_df.assembler == assembler) & (global_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        stats_per_distribution=stats_per_distribution.append(row, ignore_index=True)
    stats_per_distribution = stats_per_distribution.set_index('Assembler')
    stats_per_distribution.to_csv("Tables/Results/{} - Global metrics per assembler.csv".format(distribution))
    print(distribution)
    display(stats_per_distribution)

## Reference-based metrics

### Load data

In [ ]:
report_glob = glob.glob('../Results/*/*/report/pipeline_report_tables.json')
reference_pipeline_metrics_df = pd.DataFrame()

for pipeline_report_file in report_glob:
    report_file_name = pipeline_report_file.split('/')[-1]
    stats_run = pipeline_report_file.split('/')[-3]
    print('Processing {0} data from {1}...'.format(report_file_name, stats_run))
    
    with open(pipeline_report_file) as _fh:
        json_report = json.load(_fh)
        for sample in json_report.keys():
            for reference, data in json_report[sample]['ReferenceTables'].items():
                for row in data:
                    for item in row:
                        reference_pipeline_metrics_df = reference_pipeline_metrics_df.append({'run': stats_run,
                                                                                        'sample': sample,
                                                                                        'assembler': item['assembler'],
                                                                                        'reference': reference,
                                                                                        'LSA': item['contiguity'],
                                                                                        'breadth_of_coverage': item['breadth_of_coverage'],
                                                                                        'multiplicity': item['multiplicity'],
                                                                                        'validity': item['validity'],
                                                                                        'parsimony': item['parsimony'],
                                                                                        'identity': item['identity'],
                                                                                        'lowest_identity': item['lowest_identity'],
                                                                                        'L90': item['L90'],
                                                                                        'contigs': item['aligned_contigs'],
                                                                                        'NA50': item['NA50'],
                                                                                        'NG50': item['NG50'],
                                                                                        'basepairs': item['aligned_basepairs'],
                                                                                        'Ns': item['Ns'],
                                                                                        'misassembled contigs': item['misassembled_contigs'],
                                                                                        'misassembly events': item['misassembly_events']},
                                                                                       ignore_index=True)

reference_pipeline_metrics_df['distribution'] = np.where(reference_pipeline_metrics_df['sample'].isin(log_distributed), 'Log', 'Even')
reference_pipeline_metrics_df['type'] = np.where(reference_pipeline_metrics_df['assembler'].isin(genomic_assemblers), 'Genomic', 'Metagenomic')
reference_pipeline_metrics_df['algorythm'] = np.where(reference_pipeline_metrics_df['assembler'].isin(single_kmer), 'Single k-mer De Bruijn graph', 'Multiple k-mer De Bruijn graph')
reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events', 'multiplicity','validity','parsimony','identity','lowest_identity']] = reference_pipeline_metrics_df[['contigs','basepairs','L90','Ns','NA50','NG50','misassembled contigs', 'misassembly events','multiplicity','validity','parsimony','identity','lowest_identity']].apply(pd.to_numeric)
display(reference_pipeline_metrics_df)

In [ ]:
reference_pipeline_metrics_df.dtypes

### Save data

In [ ]:
reference_pipeline_metrics_df.to_csv("Tables/Results/Reference metrics.csv")

### Reference statistics per assembler type

In [ ]:
ref_stats_per_assembler_type = pd.DataFrame()
for assembly_type in reference_pipeline_metrics_df.type.unique():
    row={'Assembler Type': assembly_type}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.type == assembly_type].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler_type=ref_stats_per_assembler_type.append(row, ignore_index=True)
ref_stats_per_assembler_type = ref_stats_per_assembler_type.set_index('Assembler Type')
ref_stats_per_assembler_type.to_csv("Tables/Results/Reference metrics per assembler type.csv")

In [ ]:
ref_stats_per_assembler_type

### Reference stats per assembly algorithm

In [ ]:
ref_stats_per_assembler_algo = pd.DataFrame()
for assembly_algo in reference_pipeline_metrics_df.algorythm.unique():
    row={'Assembler Algorithm': assembly_algo}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.algorythm == assembly_algo].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler_algo=ref_stats_per_assembler_algo.append(row, ignore_index=True)
ref_stats_per_assembler_algo = ref_stats_per_assembler_algo.set_index('Assembler Algorithm')
ref_stats_per_assembler_algo.to_csv("Tables/Results/Reference metrics per assembler algorithm.csv")

In [ ]:
ref_stats_per_assembler_algo

### Reference stats per assembler

In [ ]:
ref_stats_per_assembler = pd.DataFrame()
for assembler in reference_pipeline_metrics_df.assembler.unique():
    row={'Assembler': assembler, 
         'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
         'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
    lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_assembler=ref_stats_per_assembler.append(row, ignore_index=True)
ref_stats_per_assembler = ref_stats_per_assembler.set_index('Assembler')
ref_stats_per_assembler.to_csv("Tables/Results/Reference metrics per assembler.csv")

In [ ]:
ref_stats_per_assembler

### Reference stats per reference replicon

#### Per assembler for each reference replicon

In [ ]:
reference_pipeline_metrics_df.reference.unique()

In [ ]:
for reference in reference_pipeline_metrics_df.reference.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.reference == reference)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(reference))
    print(reference)
    display(ref_stats_per_reference_per_assembler)

#### Per reference replicon for all assemblers - Even only!

In [ ]:
ref_stats_per_reference_all_assemblers = pd.DataFrame()
for reference in reference_pipeline_metrics_df.reference.unique():
    row={'Reference': reference}
    lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.reference == reference) & (reference_pipeline_metrics_df.distribution == 'Even')].describe()
    for column in lala.columns:
        mean = lala.loc['mean',column]
        minimum = lala.loc['min',column]
        maximum = lala.loc['max',column]
        row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    ref_stats_per_reference_all_assemblers=ref_stats_per_reference_all_assemblers.append(row, ignore_index=True)
ref_stats_per_reference_all_assemblers = ref_stats_per_reference_all_assemblers.set_index('Reference')
ref_stats_per_reference_all_assemblers.to_csv("Tables/Results/Reference metrics per replicon.csv")
display(ref_stats_per_reference_all_assemblers)

### Reference stats per distribution

In [ ]:
for distribution in reference_pipeline_metrics_df.distribution.unique():
    ref_stats_per_reference_per_assembler = pd.DataFrame()
    for assembler in reference_pipeline_metrics_df.assembler.unique():
        row={'Assembler': assembler, 
             'Type':reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].type.unique()[0],
             'Algorithm': reference_pipeline_metrics_df[reference_pipeline_metrics_df.assembler == assembler].algorythm.unique()[0]}
        lala=reference_pipeline_metrics_df[(reference_pipeline_metrics_df.assembler == assembler) & (reference_pipeline_metrics_df.distribution == distribution)].describe()
        for column in lala.columns:
            mean = lala.loc['mean',column]
            minimum = lala.loc['min',column]
            maximum = lala.loc['max',column]
            row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
        ref_stats_per_reference_per_assembler=ref_stats_per_reference_per_assembler.append(row, ignore_index=True)
    ref_stats_per_reference_per_assembler = ref_stats_per_reference_per_assembler.set_index('Assembler')
    ref_stats_per_reference_per_assembler.to_csv("Tables/Results/{} - Reference metrics per assembler.csv".format(distribution))
    print(distribution)
    display(ref_stats_per_reference_per_assembler)

### Genomes vs Plasmids

In [ ]:
ref_stats_per_replicon_type = pd.DataFrame()
# Genome
row={'Replicon': "Genome"}
lala=reference_pipeline_metrics_df[~reference_pipeline_metrics_df['reference'].str.contains("plasmid")].describe()
for column in lala.columns:
    mean = lala.loc['mean',column]
    std = lala.loc['std',column]
    row[column] = "{}(± {})".format(round(mean, 2), round(std, 2))
ref_stats_per_replicon_type =ref_stats_per_replicon_type .append(row, ignore_index=True)

# Plasmid
row={'Replicon': "Plasmid"}
lala=reference_pipeline_metrics_df[reference_pipeline_metrics_df['reference'].str.contains("plasmid")].describe()
for column in lala.columns:
    mean = lala.loc['mean',column]
    minimum = lala.loc['min',column]
    maximum = lala.loc['max',column]
    row[column] = "{} [{};{}]".format(round(mean, 2),int(minimum), int(maximum))
    
ref_stats_per_replicon_type =ref_stats_per_replicon_type .append(row, ignore_index=True)
ref_stats_per_replicon_type  = ref_stats_per_replicon_type .set_index('Replicon')
ref_stats_per_replicon_type.to_csv("Tables/Results/Reference metrics per replicon type.csv")
display(ref_stats_per_replicon_type)